# Overview of Hostile Post Detection in Hindi
- **Goals:** To predict the label set of the set of post collected from Twitter and facebook.

- **Traning data:** post with their label.

- **Tesing data:** set of posts.
- **Types of post:** 
1. *Fake News:* A claim or information that is verified to be not true.
2. *Hate Speech:* A post targeting a specific group of people based on their ethnicity, religious beliefs, geographical belonging, race, etc., with malicious intentions of spreading hate or encouraging violence.
3. *Offensive:* A post containing profanity, impolite, rude, or vulgar language to insult a targeted individual or group.
4. *Defamation:* A mis-information regarding an individual or group.
5. *Non-hostile:* A post without any hostility.

## Exploring dataset

In [1]:
# importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.multiclass import OneVsRestClassifier

In [2]:
#load data from train and validation file

data = pd.read_csv("dataset.csv")
data2 = pd.read_csv("validation.csv")

In [3]:
data.head()

,Unique ID,Post,Labels Set,Unnamed: 3,Unnamed: 4
0,1,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्क...,"hate,offensive",NaN,NaN
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile,NaN,NaN
2,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...",non-hostile,NaN,NaN
3,4,@prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं...,"defamation,offensive",NaN,NaN
4,5,#unlock4guidelines - अनलॉक-4 के लिए गाइडलाइन्स...,non-hostile,NaN,NaN


In [4]:
data2.head()

,Unique ID,Post,Labels Set
0,1,दृढ़ इच्छा शक्ति से परिपूर्ण प्रणबदा के लिए दे...,non-hostile
1,2,भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं ...,defamation
2,3,कोरोना से निपटने की तैयारी / दिल्ली में 10 हजा...,non-hostile
3,4,गवर्नर कॉन्फ्रेंस में PM मोदी बोले- शिक्षा नीत...,non-hostile
4,5,"यूपी: गाजीपुर में Toilet घोटाला, प्रधान व सचिव...",non-hostile


In [5]:
pd.set_option('display.max_colwidth', -1)

c:\users\himanshu mishra\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5728 entries, 0 to 5727
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unique ID   5728 non-null   int64  
 1   Post        5728 non-null   object 
 2   Labels Set  5728 non-null   object 
 3   Unnamed: 3  0 non-null      float64
 4   Unnamed: 4  0 non-null      float64
dtypes: float64(2), int64(1), object(2)
memory usage: 223.9+ KB


In [7]:
#removing extra columns from train file

data=data.drop(data.columns[[0,3,4]], axis=1)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5728 entries, 0 to 5727
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Post        5728 non-null   object
 1   Labels Set  5728 non-null   object
dtypes: object(2)
memory usage: 89.6+ KB


In [9]:
data.head()

,Post,Labels Set
0,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है \n\n🙏 जय श्री राम 🙏,"hate,offensive"
1,"सरकार हमेशा से किसानों की कमाई को बढ़ाने के लिए नई-नई स्कीमें लाती रहती है, ताकि उन पर ज्यादा आर्थिक बोझ न पड़े.\n\nhttps://t.co/8iy2MJSBAs",non-hostile
2,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील दीपेश को सुशांत की हत्या के दिन ही क्यों याद आई? देखिए 'पूछता है भारत' अर्नब के साथ रिपब्लिक भारत पर #LIVE : https://t.co/G945HvzM0Z https://t.co/KfH7xF1IdM",non-hostile
3,@prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते है की संविधान सबको बराबर अधिकार देता है। सच्चाई यह है कि यह बराबर अधिकार नहीं देता है।,"defamation,offensive"
4,#unlock4guidelines - अनलॉक-4 के लिए गाइडलाइन्स जारी\n\n- 7 सितंबर से देशभर में मेट्रो सेवा शुरु होगी\n- 21 सितंबर के बाद रैलियों और बाकी फंक्शन में 100 लोगों को इजाजत\n- कंटेनमेंट जोन में कोई छूट नहीं\n- सिनेमाहॉल अभी बंद रहेंगे\n- 9 से 12वीं के छात्र 21 सितंबर के बाद स्कूल जा सकेंगे. https://t.co/4e6lysg0VR,non-hostile


In [10]:
data.tail()

,Post,Labels Set
5723,उदितराज जी हिम्मत जुटा कर उस नेता के लिए कुछ कह दीजिये जिसके स्वागत में मुंबई स्टेशन पर पाकिस्तान जिंदाबाद के नारे लगे विरोध कर दें मान्य होगा की आप देशभक्त हैं वरना अनर्गल प्रलाप न करें,hate
5724,"उप्र: पीएम रिपोर्ट में खुलासा: हार्टअटैक से हुई थी पूर्व विधायक निर्वेंद्र की मौत; शरीर पर नहीं मिले चोट के निशान, बेटे ने पिटाई कर हत्या का दावा किया था\nhttps://t.co/YaFEzPhTDC #NirvendraMishra @Uppolice @UPGovt",non-hostile
5725,"नौकरी गंवा चुके दोस्त की मदद: नगद के बजाए गिफ्ट कार्ड देने के बारे में सोचें, चुका सकते हैं नेटफ्लिक्स और जिम की फीस; हालचाल जानें, लेकिन ज्यादा नहीं https://t.co/l6kFR8eiBf #HelpYourFriend #Jobless #COVID19 https://t.co/YOKsAyndNa",non-hostile
5726,बंगाल में हिन्दू मरे हैं इसलिए मुझे कोई फर्क नहीं पड़ता –#रविश कुमार\n\n,fake
5727,रक्षा मंत्री @rajnathsingh ने पूर्व राष्ट्रपति #PranabMukherjee के निधन पर शोक व्यक्त किया\n\nविवरण : https://t.co/aLE7aVtWy9,non-hostile


In [11]:
#Calculating the null values in each columns

data2.isnull().sum()

Unique ID     0
Post          0
Labels Set    0
dtype: int64

In [12]:
data.head()

,Post,Labels Set
0,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है \n\n🙏 जय श्री राम 🙏,"hate,offensive"
1,"सरकार हमेशा से किसानों की कमाई को बढ़ाने के लिए नई-नई स्कीमें लाती रहती है, ताकि उन पर ज्यादा आर्थिक बोझ न पड़े.\n\nhttps://t.co/8iy2MJSBAs",non-hostile
2,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील दीपेश को सुशांत की हत्या के दिन ही क्यों याद आई? देखिए 'पूछता है भारत' अर्नब के साथ रिपब्लिक भारत पर #LIVE : https://t.co/G945HvzM0Z https://t.co/KfH7xF1IdM",non-hostile
3,@prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते है की संविधान सबको बराबर अधिकार देता है। सच्चाई यह है कि यह बराबर अधिकार नहीं देता है।,"defamation,offensive"
4,#unlock4guidelines - अनलॉक-4 के लिए गाइडलाइन्स जारी\n\n- 7 सितंबर से देशभर में मेट्रो सेवा शुरु होगी\n- 21 सितंबर के बाद रैलियों और बाकी फंक्शन में 100 लोगों को इजाजत\n- कंटेनमेंट जोन में कोई छूट नहीं\n- सिनेमाहॉल अभी बंद रहेंगे\n- 9 से 12वीं के छात्र 21 सितंबर के बाद स्कूल जा सकेंगे. https://t.co/4e6lysg0VR,non-hostile


In [13]:
#removing url,hashtag and mentions from train dataset

data['cleanPost'] = data['Post'].str.replace('http\S+|www.\S+|#\S+|@\S+', '', case=False)

In [14]:
#removing url,hashtag and mentions from validation dataset

data2['cleanPost'] = data2['Post'].str.replace('http\S+|www.\S+|#\S+|@\S+', '', case=False)

In [15]:
data.head()

,Post,Labels Set,cleanPost
0,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है \n\n🙏 जय श्री राम 🙏,"hate,offensive",मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है \n\n🙏 जय श्री राम 🙏
1,"सरकार हमेशा से किसानों की कमाई को बढ़ाने के लिए नई-नई स्कीमें लाती रहती है, ताकि उन पर ज्यादा आर्थिक बोझ न पड़े.\n\nhttps://t.co/8iy2MJSBAs",non-hostile,"सरकार हमेशा से किसानों की कमाई को बढ़ाने के लिए नई-नई स्कीमें लाती रहती है, ताकि उन पर ज्यादा आर्थिक बोझ न पड़े.\n\n"
2,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील दीपेश को सुशांत की हत्या के दिन ही क्यों याद आई? देखिए 'पूछता है भारत' अर्नब के साथ रिपब्लिक भारत पर #LIVE : https://t.co/G945HvzM0Z https://t.co/KfH7xF1IdM",non-hostile,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील दीपेश को सुशांत की हत्या के दिन ही क्यों याद आई? देखिए 'पूछता है भारत' अर्नब के साथ रिपब्लिक भारत पर :"
3,@prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते है की संविधान सबको बराबर अधिकार देता है। सच्चाई यह है कि यह बराबर अधिकार नहीं देता है।,"defamation,offensive",साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते है की संविधान सबको बराबर अधिकार देता है। सच्चाई यह है कि यह बराबर अधिकार नहीं देता है।
4,#unlock4guidelines - अनलॉक-4 के लिए गाइडलाइन्स जारी\n\n- 7 सितंबर से देशभर में मेट्रो सेवा शुरु होगी\n- 21 सितंबर के बाद रैलियों और बाकी फंक्शन में 100 लोगों को इजाजत\n- कंटेनमेंट जोन में कोई छूट नहीं\n- सिनेमाहॉल अभी बंद रहेंगे\n- 9 से 12वीं के छात्र 21 सितंबर के बाद स्कूल जा सकेंगे. https://t.co/4e6lysg0VR,non-hostile,- अनलॉक-4 के लिए गाइडलाइन्स जारी\n\n- 7 सितंबर से देशभर में मेट्रो सेवा शुरु होगी\n- 21 सितंबर के बाद रैलियों और बाकी फंक्शन में 100 लोगों को इजाजत\n- कंटेनमेंट जोन में कोई छूट नहीं\n- सिनेमाहॉल अभी बंद रहेंगे\n- 9 से 12वीं के छात्र 21 सितंबर के बाद स्कूल जा सकेंगे.


In [16]:
data2.head()

,Unique ID,Post,Labels Set,cleanPost
0,1,दृढ़ इच्छा शक्ति से परिपूर्ण प्रणबदा के लिए देशहित सर्वोच्च रहा।\n\nउनका निधन हम सब के लिए अपूरणीय क्षति है।\nईश्वर दिवंगत आत्मा को अपने श्रीचरणों में स्थान दें। शोक संतप्त परिजनों के प्रति संवेदनाएं।\nऊं शांति!!!,non-hostile,दृढ़ इच्छा शक्ति से परिपूर्ण प्रणबदा के लिए देशहित सर्वोच्च रहा।\n\nउनका निधन हम सब के लिए अपूरणीय क्षति है।\nईश्वर दिवंगत आत्मा को अपने श्रीचरणों में स्थान दें। शोक संतप्त परिजनों के प्रति संवेदनाएं।\nऊं शांति!!!
1,2,भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं जहां मैं रहती हूं वहां मेरी जासूसी करा रहें है उसकी जासूस की पहचान मुझे अच्छी तरह है rss बीजेपी वाले की जासूस दिल्ली में कौन है उत्तर प्रदेश में कौन है हरियाणा राजस्थान में कौन है सबकी पहचान है मुझे मेरी नजर से बच नहीं सकते हो,defamation,भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं जहां मैं रहती हूं वहां मेरी जासूसी करा रहें है उसकी जासूस की पहचान मुझे अच्छी तरह है rss बीजेपी वाले की जासूस दिल्ली में कौन है उत्तर प्रदेश में कौन है हरियाणा राजस्थान में कौन है सबकी पहचान है मुझे मेरी नजर से बच नहीं सकते हो
2,3,"कोरोना से निपटने की तैयारी / दिल्ली में 10 हजार बेड वाला दुनिया का सबसे बड़ा कोविड केयर सेंटर शुरू, राजनाथ-शाह ने डीआरडीओ के 1 हजार बेड वाले सेंटर का भी उद्घाटन किया\nhttps://t.co/9rlQowAsFh #Delhi @ArvindKejriwal @rajnathsingh @AmitShah @DRDO_India @WHO @crpfindia @ITBP_official",non-hostile,"कोरोना से निपटने की तैयारी / दिल्ली में 10 हजार बेड वाला दुनिया का सबसे बड़ा कोविड केयर सेंटर शुरू, राजनाथ-शाह ने डीआरडीओ के 1 हजार बेड वाले सेंटर का भी उद्घाटन किया\n"
3,4,गवर्नर कॉन्फ्रेंस में PM मोदी बोले- शिक्षा नीति में सरकार का दखल कम होना चाहिए\nhttps://t.co/ZvKgxk6dbd,non-hostile,गवर्नर कॉन्फ्रेंस में PM मोदी बोले- शिक्षा नीति में सरकार का दखल कम होना चाहिए\n
4,5,"यूपी: गाजीपुर में Toilet घोटाला, प्रधान व सचिव ने किया लाखों का गबन, मुर्दों के नाम पर बनवा डाले शौचालय\n\n#UP\nhttps://t.co/hxM1uNNmX2",non-hostile,"यूपी: गाजीपुर में Toilet घोटाला, प्रधान व सचिव ने किया लाखों का गबन, मुर्दों के नाम पर बनवा डाले शौचालय\n\n\n"


In [17]:
#removing post columns from train 

columns_titles = ["cleanPost","Labels Set","Post"]
data=data.reindex(columns=columns_titles)

In [18]:
data=data.drop(data.columns[[2]], axis=1)

In [19]:
#removing post column from validation

columns_titles = ["cleanPost","Labels Set","Post"]
data2=data2.reindex(columns=columns_titles)
data2=data2.drop(data2.columns[[2]], axis=1)

In [20]:
data.head()

,cleanPost,Labels Set
0,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है \n\n🙏 जय श्री राम 🙏,"hate,offensive"
1,"सरकार हमेशा से किसानों की कमाई को बढ़ाने के लिए नई-नई स्कीमें लाती रहती है, ताकि उन पर ज्यादा आर्थिक बोझ न पड़े.\n\n",non-hostile
2,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील दीपेश को सुशांत की हत्या के दिन ही क्यों याद आई? देखिए 'पूछता है भारत' अर्नब के साथ रिपब्लिक भारत पर :",non-hostile
3,साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते है की संविधान सबको बराबर अधिकार देता है। सच्चाई यह है कि यह बराबर अधिकार नहीं देता है।,"defamation,offensive"
4,- अनलॉक-4 के लिए गाइडलाइन्स जारी\n\n- 7 सितंबर से देशभर में मेट्रो सेवा शुरु होगी\n- 21 सितंबर के बाद रैलियों और बाकी फंक्शन में 100 लोगों को इजाजत\n- कंटेनमेंट जोन में कोई छूट नहीं\n- सिनेमाहॉल अभी बंद रहेंगे\n- 9 से 12वीं के छात्र 21 सितंबर के बाद स्कूल जा सकेंगे.,non-hostile


In [21]:
data2.head()

,cleanPost,Labels Set
0,दृढ़ इच्छा शक्ति से परिपूर्ण प्रणबदा के लिए देशहित सर्वोच्च रहा।\n\nउनका निधन हम सब के लिए अपूरणीय क्षति है।\nईश्वर दिवंगत आत्मा को अपने श्रीचरणों में स्थान दें। शोक संतप्त परिजनों के प्रति संवेदनाएं।\nऊं शांति!!!,non-hostile
1,भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं जहां मैं रहती हूं वहां मेरी जासूसी करा रहें है उसकी जासूस की पहचान मुझे अच्छी तरह है rss बीजेपी वाले की जासूस दिल्ली में कौन है उत्तर प्रदेश में कौन है हरियाणा राजस्थान में कौन है सबकी पहचान है मुझे मेरी नजर से बच नहीं सकते हो,defamation
2,"कोरोना से निपटने की तैयारी / दिल्ली में 10 हजार बेड वाला दुनिया का सबसे बड़ा कोविड केयर सेंटर शुरू, राजनाथ-शाह ने डीआरडीओ के 1 हजार बेड वाले सेंटर का भी उद्घाटन किया\n",non-hostile
3,गवर्नर कॉन्फ्रेंस में PM मोदी बोले- शिक्षा नीति में सरकार का दखल कम होना चाहिए\n,non-hostile
4,"यूपी: गाजीपुर में Toilet घोटाला, प्रधान व सचिव ने किया लाखों का गबन, मुर्दों के नाम पर बनवा डाले शौचालय\n\n\n",non-hostile


In [22]:
#getting list from Labels set for train

for i in range(len(data['Labels Set'])):
    data['Labels Set'][i]=list(data['Labels Set'][i].split(','))

In [23]:
#getting list from Labels set for validation

for i in range(len(data2['Labels Set'])):
    data2['Labels Set'][i]=list(data2['Labels Set'][i].split(','))
    #print(i)

In [24]:
data.head()

,cleanPost,Labels Set
0,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है \n\n🙏 जय श्री राम 🙏,"[hate, offensive]"
1,"सरकार हमेशा से किसानों की कमाई को बढ़ाने के लिए नई-नई स्कीमें लाती रहती है, ताकि उन पर ज्यादा आर्थिक बोझ न पड़े.\n\n",[non-hostile]
2,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील दीपेश को सुशांत की हत्या के दिन ही क्यों याद आई? देखिए 'पूछता है भारत' अर्नब के साथ रिपब्लिक भारत पर :",[non-hostile]
3,साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते है की संविधान सबको बराबर अधिकार देता है। सच्चाई यह है कि यह बराबर अधिकार नहीं देता है।,"[defamation, offensive]"
4,- अनलॉक-4 के लिए गाइडलाइन्स जारी\n\n- 7 सितंबर से देशभर में मेट्रो सेवा शुरु होगी\n- 21 सितंबर के बाद रैलियों और बाकी फंक्शन में 100 लोगों को इजाजत\n- कंटेनमेंट जोन में कोई छूट नहीं\n- सिनेमाहॉल अभी बंद रहेंगे\n- 9 से 12वीं के छात्र 21 सितंबर के बाद स्कूल जा सकेंगे.,[non-hostile]


In [25]:
data2.head()

,cleanPost,Labels Set
0,दृढ़ इच्छा शक्ति से परिपूर्ण प्रणबदा के लिए देशहित सर्वोच्च रहा।\n\nउनका निधन हम सब के लिए अपूरणीय क्षति है।\nईश्वर दिवंगत आत्मा को अपने श्रीचरणों में स्थान दें। शोक संतप्त परिजनों के प्रति संवेदनाएं।\nऊं शांति!!!,[non-hostile]
1,भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं जहां मैं रहती हूं वहां मेरी जासूसी करा रहें है उसकी जासूस की पहचान मुझे अच्छी तरह है rss बीजेपी वाले की जासूस दिल्ली में कौन है उत्तर प्रदेश में कौन है हरियाणा राजस्थान में कौन है सबकी पहचान है मुझे मेरी नजर से बच नहीं सकते हो,[defamation]
2,"कोरोना से निपटने की तैयारी / दिल्ली में 10 हजार बेड वाला दुनिया का सबसे बड़ा कोविड केयर सेंटर शुरू, राजनाथ-शाह ने डीआरडीओ के 1 हजार बेड वाले सेंटर का भी उद्घाटन किया\n",[non-hostile]
3,गवर्नर कॉन्फ्रेंस में PM मोदी बोले- शिक्षा नीति में सरकार का दखल कम होना चाहिए\n,[non-hostile]
4,"यूपी: गाजीपुर में Toilet घोटाला, प्रधान व सचिव ने किया लाखों का गबन, मुर्दों के नाम पर बनवा डाले शौचालय\n\n\n",[non-hostile]


In [28]:
multilabel = MultiLabelBinarizer()


In [29]:
y = multilabel.fit_transform(data['Labels Set'])
y

array([[0, 0, 1, 0, 1],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       ...,
       [0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0]])

In [30]:
multilabel.classes_


array(['defamation', 'fake', 'hate', 'non-hostile', 'offensive'],
      dtype=object)

In [31]:
#np array dataframe for train

df_1=pd.DataFrame(y, columns=multilabel.classes_)
df_1

,defamation,fake,hate,non-hostile,offensive
0,0,0,1,0,1
1,0,0,0,1,0
2,0,0,0,1,0
3,1,0,0,0,1
4,0,0,0,1,0
...,...,...,...,...,...
5723,0,0,1,0,0
5724,0,0,0,1,0
5725,0,0,0,1,0
5726,0,1,0,0,0


In [32]:
#np array dataframe for validation

multilabel = MultiLabelBinarizer()
y = multilabel.fit_transform(data2['Labels Set'])
df_2=pd.DataFrame(y, columns=multilabel.classes_)
df_2

,defamation,fake,hate,non-hostile,offensive
0,0,0,0,1,0
1,1,0,0,0,0
2,0,0,0,1,0
3,0,0,0,1,0
4,0,0,0,1,0
...,...,...,...,...,...
806,0,0,1,0,0
807,1,0,0,0,0
808,0,0,0,1,0
809,0,0,0,1,0


In [33]:
df_1.head()

,defamation,fake,hate,non-hostile,offensive
0,0,0,1,0,1
1,0,0,0,1,0
2,0,0,0,1,0
3,1,0,0,0,1
4,0,0,0,1,0


In [34]:
df_2.head()

,defamation,fake,hate,non-hostile,offensive
0,0,0,0,1,0
1,1,0,0,0,0
2,0,0,0,1,0
3,0,0,0,1,0
4,0,0,0,1,0


In [35]:
result = pd.concat([data, df_1], axis=1, sort=False)
result2 = pd.concat([data2,df_2],axis=1,sort=False)

In [36]:
result = result.rename({'non-hostile': 'non_hostile'}, axis=1)
result2 = result2.rename({'non-hostile': 'non_hostile'}, axis=1)

In [39]:
"""
f=open("stopWords.txt","r")
stopWords=set(f.read().split('\n'))

index=0

for i in result['cleanPost']:
    temp=''
    words=i.split(' ')
    for j in words:
        if j not in stopWords:
            temp+=j
            temp+=' '
    result['cleanPost'][index]=temp
    index+=1

""""

SyntaxError: EOL while scanning string literal (<ipython-input-39-49f1c70d63a8>, line 17)

In [40]:
""""
f=open("stopWords.txt","r")
stopWords=set(f.read().split('\n'))

index=0

for i in result2['cleanPost']:
    temp=''
    words=i.split(' ')
    for j in words:
        if j not in stopWords:
            temp+=j
            temp+=' '
    result2['cleanPost'][index]=temp
    index+=1

""""

SyntaxError: EOL while scanning string literal (<ipython-input-40-9f8f6cf857aa>, line 17)

In [37]:
# train dataframe 
result

,cleanPost,Labels Set,defamation,fake,hate,non_hostile,offensive
0,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है \n\n🙏 जय श्री राम 🙏,"[hate, offensive]",0,0,1,0,1
1,"सरकार हमेशा से किसानों की कमाई को बढ़ाने के लिए नई-नई स्कीमें लाती रहती है, ताकि उन पर ज्यादा आर्थिक बोझ न पड़े.\n\n",[non-hostile],0,0,0,1,0
2,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील दीपेश को सुशांत की हत्या के दिन ही क्यों याद आई? देखिए 'पूछता है भारत' अर्नब के साथ रिपब्लिक भारत पर :",[non-hostile],0,0,0,1,0
3,साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते है की संविधान सबको बराबर अधिकार देता है। सच्चाई यह है कि यह बराबर अधिकार नहीं देता है।,"[defamation, offensive]",1,0,0,0,1
4,- अनलॉक-4 के लिए गाइडलाइन्स जारी\n\n- 7 सितंबर से देशभर में मेट्रो सेवा शुरु होगी\n- 21 सितंबर के बाद रैलियों और बाकी फंक्शन में 100 लोगों को इजाजत\n- कंटेनमेंट जोन में कोई छूट नहीं\n- सिनेमाहॉल अभी बंद रहेंगे\n- 9 से 12वीं के छात्र 21 सितंबर के बाद स्कूल जा सकेंगे.,[non-hostile],0,0,0,1,0
...,...,...,...,...,...,...,...
5723,उदितराज जी हिम्मत जुटा कर उस नेता के लिए कुछ कह दीजिये जिसके स्वागत में मुंबई स्टेशन पर पाकिस्तान जिंदाबाद के नारे लगे विरोध कर दें मान्य होगा की आप देशभक्त हैं वरना अनर्गल प्रलाप न करें,[hate],0,0,1,0,0
5724,"उप्र: पीएम रिपोर्ट में खुलासा: हार्टअटैक से हुई थी पूर्व विधायक निर्वेंद्र की मौत; शरीर पर नहीं मिले चोट के निशान, बेटे ने पिटाई कर हत्या का दावा किया था\n",[non-hostile],0,0,0,1,0
5725,"नौकरी गंवा चुके दोस्त की मदद: नगद के बजाए गिफ्ट कार्ड देने के बारे में सोचें, चुका सकते हैं नेटफ्लिक्स और जिम की फीस; हालचाल जानें, लेकिन ज्यादा नहीं",[non-hostile],0,0,0,1,0
5726,बंगाल में हिन्दू मरे हैं इसलिए मुझे कोई फर्क नहीं पड़ता – कुमार\n\n,[fake],0,1,0,0,0


In [38]:
#validation dataframe
result2

,cleanPost,Labels Set,defamation,fake,hate,non_hostile,offensive
0,दृढ़ इच्छा शक्ति से परिपूर्ण प्रणबदा के लिए देशहित सर्वोच्च रहा।\n\nउनका निधन हम सब के लिए अपूरणीय क्षति है।\nईश्वर दिवंगत आत्मा को अपने श्रीचरणों में स्थान दें। शोक संतप्त परिजनों के प्रति संवेदनाएं।\nऊं शांति!!!,[non-hostile],0,0,0,1,0
1,भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं जहां मैं रहती हूं वहां मेरी जासूसी करा रहें है उसकी जासूस की पहचान मुझे अच्छी तरह है rss बीजेपी वाले की जासूस दिल्ली में कौन है उत्तर प्रदेश में कौन है हरियाणा राजस्थान में कौन है सबकी पहचान है मुझे मेरी नजर से बच नहीं सकते हो,[defamation],1,0,0,0,0
2,"कोरोना से निपटने की तैयारी / दिल्ली में 10 हजार बेड वाला दुनिया का सबसे बड़ा कोविड केयर सेंटर शुरू, राजनाथ-शाह ने डीआरडीओ के 1 हजार बेड वाले सेंटर का भी उद्घाटन किया\n",[non-hostile],0,0,0,1,0
3,गवर्नर कॉन्फ्रेंस में PM मोदी बोले- शिक्षा नीति में सरकार का दखल कम होना चाहिए\n,[non-hostile],0,0,0,1,0
4,"यूपी: गाजीपुर में Toilet घोटाला, प्रधान व सचिव ने किया लाखों का गबन, मुर्दों के नाम पर बनवा डाले शौचालय\n\n\n",[non-hostile],0,0,0,1,0
...,...,...,...,...,...,...,...
806,चंद फेंके हुए टुकड़ों के लिए हमेशा ही देश से गद्दारी को भौंकने तैयार\nड्रग्स के नशे में पूरी तरह खोखले हो चुके दाऊद के गुलाम देशद्रोही गद्दार फिल्मी भांडों के भोंकने को भी मीडिया ब्रेकिंग न्यूज़ बना\nइन दो कौड़ी के बिकाऊ नशेड़ी गद्दार कुत्तों को हीरो बना देती है\nदेश की विडंबना😔,[hate],0,0,1,0,0
807,आँखों ही आँखों में इशारा हो गया \nबैठे बैठे जीने का सहारा हो गया 😉😉😉\nनज़रों की इस मारक क्षमता को क्या कहगे। 👌,[defamation],1,0,0,0,0
808,"औरंगाबाद में फिर धीरे-धीरे पांव पसार रहा कोरोना संक्रमण, पिछले 6 दिनों में मिले इतने नए मरीज\n\n \n\n",[non-hostile],0,0,0,1,0
809,"बाइस्कोप: कादर के संवादों का कमाल, डिंपल की कलम का धमाल और तीन बत्ती के जग्गू का नाना संग असली बवाल\n \n \n\n",[non-hostile],0,0,0,1,0


In [39]:
data = data.rename({'Labels Set': 'Labels'}, axis=1)
data2 = data2.rename({'Labels Set': 'Labels'}, axis=1)

In [63]:
#y = result.offensive.values

In [64]:
#x_train, x_test, y_train, y_test = train_test_split(result.cleanPost.values, y, stratify=y,random_state=1,test_size=0.2, shuffle=True)

In [40]:
#loading train and test data 

x_train = data.cleanPost.values
x_test = data2.cleanPost.values
y_train = data.Labels.values
y_test = data2.Labels.values

In [41]:
y_train

array([list(['hate', 'offensive']), list(['non-hostile']),
       list(['non-hostile']), ..., list(['non-hostile']), list(['fake']),
       list(['non-hostile'])], dtype=object)

In [42]:
from sklearn.feature_extraction.text import CountVectorizer

In [43]:
#vectorizing data
vectorizer = CountVectorizer(binary=True)

vectorizer.fit(list(x_train) + list(x_test))


x_train_vec = vectorizer.transform(x_train)
x_test_vec = vectorizer.transform(x_test)


In [44]:
from sklearn import svm
# classify using support vector classifier
svm1 = svm.SVC(kernel = 'linear', probability=True,C=0.01,gamma = 1)

# fit the SVC model based on the given training data
prob = svm1.fit(x_train_vec, y_train).predict_proba(x_test_vec)

# perform classification and prediction on samples in x_test
y_pred_svm1 = svm1.predict(x_test_vec)


ValueError: You appear to be using a legacy multi-label data representation. Sequence of sequences are no longer supported; use a binary array or sparse matrix instead - the MultiLabelBinarizer transformer can convert to this format.

In [45]:
from sklearn.metrics import accuracy_score
print("Accuracy score for SVC is: ", accuracy_score(y_test, y_pred_svm1) * 100, '%')

NameError: name 'y_pred_svm1' is not defined

In [46]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_svm1))

NameError: name 'y_pred_svm1' is not defined

In [47]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=400)

rfc.fit(x_train_vec,y_train)

y_pred_rfc = rfc.predict(x_test_vec)

print("Accuracy score for Random Forest is: ", accuracy_score(y_test, y_pred_rfc) * 100, '%')

ValueError: Unknown label type: 'unknown'

In [158]:
print(y_pred_rfc)

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 1 0]
 ...
 [0 0 0 1 0]
 [0 0 0 0 0]
 [0 0 0 1 0]]


In [165]:
print(classification_report(y_test, y_pred_rfc))

              precision    recall  f1-score   support

           0       0.75      0.81      0.78       376
           1       0.82      0.77      0.80       435

    accuracy                           0.79       811
   macro avg       0.79      0.79      0.79       811
weighted avg       0.79      0.79      0.79       811



In [ ]:
#SVM Result
#defamation = 90.50554870530209 %
#fake = 80.27127003699137 %
#hate = 86.43649815043156 %
#non-hostile = 74.9691738594328 %
#offensive = 87.2996300863132 %

#Random Forest Result
#defamation = 90.13563501849569 %
#fake = 82.98397040690506 %
#hate = 86.92971639950679 %
#non-hostile = 80.14796547472255 %
#offensive =  87.79284833538841 %